In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ndvi-sa-week2/hacktest.csv
/kaggle/input/ndvi-sa-week2/hacktrain.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.signal import savgol_filter

# Custom Feature Engineering Transformer
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, window_size=3):
        self.window_size = window_size
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Create rolling average features
        X_rolled = np.zeros((X.shape[0], X.shape[1]-self.window_size+1))
        for i in range(X_rolled.shape[1]):
            X_rolled[:, i] = X[:, i:i+self.window_size].mean(axis=1)
        return np.hstack([X, X_rolled])

# Custom Smoothing Transformer  
class SeriesSmoother(BaseEstimator, TransformerMixin):
    def __init__(self, window_length=5, polyorder=2):
        self.window_length = window_length
        self.polyorder = polyorder
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if X.shape[0] >= self.window_length:
            return savgol_filter(X, self.window_length, self.polyorder, axis=0)
        return X

# Data Loading and Preparation
train_df = pd.read_csv('/kaggle/input/ndvi-sa-week2/hacktrain.csv')
test_df = pd.read_csv("/kaggle/input/ndvi-sa-week2/hacktest.csv")

train_df = train_df.drop(columns=['Unnamed: 0'], errors='ignore')
test_df = test_df.drop(columns=['Unnamed: 0'], errors='ignore')

X = train_df.drop(columns=['ID', 'class'])
y = train_df['class']
X_test = test_df.drop(columns=['ID'])

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Optimized Pipeline
pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('feature_engineer', FeatureEngineer(window_size=3)),
    ('smoother', SeriesSmoother(window_length=5)),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=2000, 
                                     multi_class='multinomial',
                                     solver='lbfgs'))
])

pipeline.fit(X_train, y_train)

# Evaluation
y_val_pred = pipeline.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")

# Prediction
y_test_labels = label_encoder.inverse_transform(pipeline.predict(X_test))
submission = pd.DataFrame({'ID': test_df['ID'], 'class': y_test_labels})
submission.to_csv("submission1.csv", index=False)


Validation Accuracy: 0.8056
